Importar Librerias

In [72]:
import json
import pandas as pd
import gzip
import nltk as nlt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nlt.download("vader_lexicon")

#import utils


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [73]:
#almacenamiento de direcciones de DataSets en variables, y encapsulandolas
#en una lista para ser leida por script de limpieza
ruta_archivo1=r"C:\Users\Usuario\Desktop\ProyectoIndividual\DataSets-Steam\australian_user_reviews.json"
ruta_archivo2=r"C:\Users\Usuario\Desktop\ProyectoIndividual\DataSets-Steam\australian_users_items.json"
ruta_archivo3=r"C:\Users\Usuario\Desktop\ProyectoIndividual\DataSets-Steam\output_steam_games.json"
rutas=[ruta_archivo1,ruta_archivo2,ruta_archivo3]

Extraccion, Exploracion de datos y Eliminacion de Duplicados

Archivo  ruta_archivo1 <- "australian_user_reviews"

In [74]:
with open(ruta_archivo1, 'r', encoding='utf-8') as w:
    data = w.readlines()
    data = [eval(line.strip()) for line in data]
df1=pd.DataFrame(data)

In [58]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Verificacion de Duplicados, tomando el valor de referencia user_id

In [7]:
duplicated= df1[df1["user_id"].duplicated()]
print(duplicated)

                 user_id                                           user_url  \
456           bokkkbokkk            http://steamcommunity.com/id/bokkkbokkk   
1182          ImSeriouss            http://steamcommunity.com/id/ImSeriouss   
1456   76561198062039159  http://steamcommunity.com/profiles/76561198062...   
1477   76561198045009232  http://steamcommunity.com/profiles/76561198045...   
1746        nitr0ticwolf          http://steamcommunity.com/id/nitr0ticwolf   
...                  ...                                                ...   
17819  76561198076474887  http://steamcommunity.com/profiles/76561198076...   
17916        yolofaceguy           http://steamcommunity.com/id/yolofaceguy   
18028  76561198075591109  http://steamcommunity.com/profiles/76561198075...   
18234  76561198092022514  http://steamcommunity.com/profiles/76561198092...   
18309     SuchGayMuchWow        http://steamcommunity.com/id/SuchGayMuchWow   

                                                 re

In [8]:
# Se revisa un usuario de ejemplo
user_id = 'SuchGayMuchWow'
user_reviews = duplicated[duplicated['user_id'] == user_id]['reviews']

for review_list in user_reviews:
    for review in review_list:
        print(review['review'])

10/10 too many children with mikes.


Debido a que despues de verificar visualmente varios usuarios, que los comentarios no se repiten, se decide NO borrar las entradas duplicadas ya que son opiniones del mismo usuario sobre distintos juegos, en caso contrario se correria el siguiente codigo

Eliminacion de entradas duplicadas

In [ ]:
df1.drop_duplicates(subset=["user_id"], keep="first", inplace= True)


Se revisa la columna "reviews", para observar el tipo de dato que contiene 

In [75]:
df1["reviews"][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]



Este conjunto contiene 3 columnas y 25799 filas, sin valores nulos. Las columnas son:

   * user_id: es un identificador único para el usuario.
   * user_url: es la url del perfil del usuario en streamcommunity.
   * reviews: contiene una lista de diccionarios. Para cada usuario se tiene uno o mas diccionario con el review. Cada diccionario contiene:
        * funny: indica si alguien puso emoticón de gracioso al review.
        * posted: es la fecha de posteo del review en formato Posted April 21, 2011.
        * last_edited: es la fecha de la última edición.
        * item_id: es el identificador único del item, es decir, del juego.
        * helpful: es la estadística donde otros usuarios indican si fue útil la información.
        * recommend: es un booleano que indica si el usuario recomienda o no el juego.
        * review: es una sentencia string con los comentarios sobre el juego.



Se transforma la columna de "reviews" en un dataframe para poder trabajarlo

In [76]:
comments_df1=pd.json_normalize(df1["reviews"])
comments_df1.head(5)

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Se crea nuevo DataFrame, con cada registro de reviews, dividiendo en columnas el par clave-valor

In [36]:
comments_df1.shape


(25799, 3)

In [37]:
df1.shape


(25799, 3)

Se crea un nuevo dataframe, en donde se descomprime los reviews de cada usuario, concatenando el user_id de cada usuario para no perder la relacion Usuario-Comentarios de diferentes juegos.

In [88]:
index=comments_df1.shape
nuevo_df=pd.DataFrame()

for fila in range(index[0]):
    for columna in range (index[1]):
        if pd.notna(comments_df1.iloc[fila, columna]):
            df_normalizado = pd.json_normalize(comments_df1.iloc[fila, columna])
            df_normalizado['user_id'] = df1.iloc[fila]['user_id']
            nuevo_df = pd.concat([nuevo_df, df_normalizado], ignore_index=True)
            
nuevo_df

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637
...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244
59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley
59303,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley


Se eliminan las columnas irrelevantes para el Analisis de Sentimiento

In [89]:
columnas_a_eliminar=["funny","last_edited","helpful"]
nuevo_df=nuevo_df.drop(columns=columnas_a_eliminar)
nuevo_df

,posted,item_id,recommend,review,user_id
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637
...,...,...,...,...,...
59300,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244
59301,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244
59302,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley
59303,Posted July 20.,730,True,:D,LydiaMorley


Se aplica la funcion de Analisis de Sentimiento sobre la columna "reviews", colocando el resultado en una nueva columna, posteriormente se verifica

In [90]:
# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to assign values according to the scale
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Return neutral if it is empty or NaN
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 2  # Good score
        elif compound_score <= -0.05:
            return 0  # Bad score
        else:
            return 1
    else:
        return 1  # Return neutral for non-string values
    


# Convertir la columna 'review' a tipo de dato str
    
nuevo_df['review'] = nuevo_df['review'].astype(str)

# Aplicar la función get_sentiment_score a la columna 'review'
nuevo_df['sentiment_score'] = nuevo_df['review'].apply(get_sentiment_score)


In [91]:

# Imprimir el DataFrame actualizado
nuevo_df.head(5)

,posted,item_id,recommend,review,user_id,sentiment_score
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479,2
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637,2
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637,2


Se normaliza la columna "posted"

In [92]:
nuevo_df['posted'] = nuevo_df['posted'].str.replace('Posted', '')
nuevo_df

,posted,item_id,recommend,review,user_id,sentiment_score
0,"November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2
1,"July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479,2
2,"April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
3,"June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637,2
4,"September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637,2
...,...,...,...,...,...,...
59300,July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244,2
59301,July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244,2
59302,July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,2
59303,July 20.,730,True,:D,LydiaMorley,2


In [93]:
# Reemplazar los valores no válidos por NaN
nuevo_df["posted"] = pd.to_datetime(nuevo_df["posted"], errors='coerce')
# Filtrar el DataFrame para obtener los registros válidos
df_filtered = nuevo_df.dropna(subset=["posted"])
# Convertir la columna "posted" a tipo datetime
df_filtered["posted"] = pd.to_datetime(df_filtered["posted"])
# Rellenando los valores faltantes en las columna'posted' con la moda
columns_fill_mode = ['posted']
for col in columns_fill_mode:
    nuevo_df[col].fillna(nuevo_df[col].mode()[0], inplace=True)
# Modelado y eliminación de la columna 'posted'
nuevo_df['posted_year'] = nuevo_df['posted'].dt.year.astype('Int64')
nuevo_df.drop('posted', axis=1, inplace=True)
nuevo_df.drop('review', axis=1, inplace=True)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_11764\2773297591.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["posted"] = pd.to_datetime(df_filtered["posted"])


In [94]:
nuevo_df.head(5)


,item_id,recommend,user_id,sentiment_score,posted_year
0,1250,True,76561197970982479,2,2011
1,22200,True,76561197970982479,2,2011
2,43110,True,76561197970982479,2,2011
3,251610,True,js41637,2,2014
4,227300,True,js41637,2,2013


In [95]:
nuevo_df.to_csv("reviews_clean.csv", index=False)